In [1]:
%matplotlib
#%matplotlib inline
import os
import csv
import fnmatch
import numpy as np
import datetime
import re 
import pandas as pd
import matplotlib.pyplot as plt
import math
import pywt
import itertools

pd.options.mode.use_inf_as_na = True

Using matplotlib backend: TkAgg


In [28]:
ipaDifficult = list()
ipaMedium = list()
ipaEasy = list()

In [6]:
def OptiKeyTypingTime(UserKeys):
    
    TimeTyping = dict()
    
    time1, t1, t2 = UserKeys[0][0].partition('+')
    startTime = datetime.datetime.strptime(re.sub('[:.T]','-',time1[:-1]), "%Y-%m-%d-%H-%M-%S-%f")
    
    time2, t1, t2 = UserKeys[-1][0].partition('+')
    endTime = datetime.datetime.strptime(re.sub('[:.T]','-',time2[:-1]), "%Y-%m-%d-%H-%M-%S-%f")
    
    TimeTyping['startTime'] = startTime
    TimeTyping['endTime'] = endTime
    
    return TimeTyping

In [7]:
def FindTrialEndTimes(KeysSelected, timeTyping):
    # function to find start and end of tasks in experiments
    timeStartEnd = list() # format of this list will be: [startTime1, endTime1/startTime2, endTime2/startTime3, ..., endTimeN]
    
    timeStartEnd.append(timeTyping['startTime'])
    
    nTrial = 1
    
    for keys in KeysSelected:
        
        if keys[1] == 'NextPhrase':
            time1, t1, t2 = keys[0].partition('+')
            endTimeTrial = datetime.datetime.strptime(re.sub('[:.T]','-',time1[:-1]), "%Y-%m-%d-%H-%M-%S-%f")
            timeStartEnd.append(endTimeTrial)
    
    
    timeStartEnd.append(timeTyping['endTime'])
    
    
    return timeStartEnd

In [8]:
# function to convert list of date and time into datetime format list
def timeConversion(timeStrList):
    timeList = list()
    for time in timeStrList:
        time1, t1, t2 = time.partition('+')
        timeList.append(datetime.datetime.strptime(re.sub('[:.T]','-',time1[:-1]), "%Y-%m-%d-%H-%M-%S-%f"))
    return timeList

In [9]:
# This function will return the datetime in items which is the closest to the date pivot
def nearestTimePoint(dates, date):
    
    for d in dates:
        if d < date:
            nearestTP = d
        else:
            continue
    try: 
        nearestTP
        nearestTPind = dates.index(nearestTP)
    except:
        nearestTP = 0
        nearestTPind = -1
        
    return nearestTP, nearestTPind

In [10]:
def CreateTimeEpochsOfTrials(TimeStartEndMixed, UserKeys):
    # function to use list of mixed start and end times of trials and keys looked at by user to create trial epochs
    
    TimeEpochTrial = dict()
    TimeEpochTrial['Start'] = list()
    TimeEpochTrial['End'] = list()
    
    # Create list of times in userKeys to be able to use function 'nearestTimePoint'
    UserKeysStrTimes = [item3[0] for item3 in UserKeys]
    UserKeysTimes = timeConversion(UserKeysStrTimes)
    
    Flag_FoundSleepKey = 0 # Flag to indicate finding sleep key
    
    n = -1
    for time in TimeStartEndMixed:
        n = n + 1
        Flag_FoundSleepKey = 0
        
        if n == 0: # first time is only start time for the first trial
            TimeEpochTrial['Start'].append(time)
            continue
        elif n == len(TimeStartEndMixed)-1: # last time is only the end time for last trial
            
            TimeEpochTrial['End'].append(time)
            
        else: # the middle elements need to be divided into start and end
            TimeEpochTrial['End'].append(time)
            
            timeCheck = time
            
            # find the time in userkeys. Keep going to the previous element till you reach start of selection of
            # nextPhrase key
            while Flag_FoundSleepKey < 1:
                
                nearestToTrialStartTime, nearestToTrialStartInd = nearestTimePoint(UserKeysTimes, timeCheck)
                indCheck = nearestToTrialStartInd
                
                if 'NextPhrase' not in UserKeys[indCheck][1]:
                    TimeEpochTrial['Start'].append(nearestToTrialStartTime)
                    Flag_FoundSleepKey = 1
                    break
                else:
                    indCheck = indCheck - 2 # 2 added instead of 1, to allow nearestTimePoint to find the one before this
                    timeCheck = UserKeysTimes[indCheck]
                    
                
    return TimeEpochTrial      
            

In [11]:
def Convert2ColumnSizesTo1(GazeLog):
    # function to convert pupilsizes from 2 columns for every pupil due to comma use instead of decimal, 
    # to proper pupil sizes
    
    PupilLogL = list()
    PupilLogR = list()
    
    PupilLogL_beforeDecimal = [item4[-5] if 'Invalid' not in item4 else 'nan' for item4 in GazeLog]
    PupilLogL_afterDecimal = [item4[-4] if 'Invalid' not in item4 else 'nan' for item4 in GazeLog]
    PupilLogR_beforeDecimal = [item4[-2] if 'Invalid' not in item4 else 'nan' for item4 in GazeLog]
    PupilLogR_afterDecimal = [item4[-1] if 'Invalid' not in item4 else 'nan' for item4 in GazeLog]
    
    for i in range(0, len(PupilLogL_beforeDecimal)):
        if 'Valid' not in PupilLogL_beforeDecimal[i] and 'Valid' not in PupilLogL_afterDecimal[i]:
            if 'nan' not in PupilLogL_beforeDecimal[i] and 'nan' not in PupilLogL_afterDecimal[i]:
                PupilLogL.append(float(PupilLogL_beforeDecimal[i]+'.'+PupilLogL_afterDecimal[i]))
            else:
                PupilLogL.append(np.nan)
        else:
            # Rarely, the pupil size is a whole number
            PupilLogL.append(np.nan) # we will ignore the row, since there is no way of automatically knowing which - 
            # right or left eye has whole number pupil size
    
    for i in range(0, len(PupilLogR_beforeDecimal)):
        if 'Valid' not in PupilLogR_beforeDecimal[i] and 'Valid' not in PupilLogR_afterDecimal[i]:
            if 'nan' not in PupilLogR_beforeDecimal[i] and 'nan' not in PupilLogR_afterDecimal[i]:
                PupilLogR.append(float(PupilLogR_beforeDecimal[i]+'.'+PupilLogR_afterDecimal[i]))
            else:
                PupilLogR.append(np.nan)
        else:
            # Rarely, the pupil size is a whole number
            PupilLogL.append(np.nan) # we will ignore the row, since there is no way of automatically knowing which - 
            # right or left eye has whole number pupil size
            
    # if one of the pupils are nan, the other one is converted too
    nPupil = -1
    for pupilL in PupilLogL:
        nPupil = nPupil + 1
        if np.isnan(pupilL):
            if nPupil < len(PupilLogR):
                if not np.isnan(PupilLogR[nPupil]):
                    PupilLogR[nPupil] = np.nan
            else:
                del PupilLogL[len(PupilLogR):]
                
    nPupil = -1
    for pupilR in PupilLogR:
        nPupil = nPupil + 1
        if np.isnan(pupilR):
            if nPupil < len(PupilLogL):
                if not np.isnan(PupilLogL[nPupil]):
                    PupilLogL[nPupil] = np.nan
            else:
                del PupilLogR[len(PupilLogL):]
                
    #print(len(PupilLogL), len(PupilLogR))
    
    return PupilLogL, PupilLogR

In [12]:
def PupilSizeFromTrialTimes(TimeTrial, TimeGazeLog, TimeInternalGazeLog, PupilSizeLogL, PupilSizeLogR):
    # find pupil sizes from the start and end time given
    
    # find start and end time in gazeLog
    timeStart, timeStartInd = nearestTimePoint(TimeGazeLog, TimeTrial[0])
    timeEnd, timeEndInd = nearestTimePoint(TimeGazeLog, TimeTrial[1])
    
    pupilSize_TrialL = PupilSizeLogL[timeStartInd: timeEndInd]
    pupilSize_TrialR = PupilSizeLogR[timeStartInd: timeEndInd]
    
    TimeInternal_Trial = TimeInternalGazeLog[timeStartInd: timeEndInd]
    
    TimeGaze_Trial = TimeGazeLog[timeStartInd: timeEndInd]
    
    return pupilSize_TrialL, pupilSize_TrialR, TimeGaze_Trial, TimeInternal_Trial
    

In [13]:
def filterBlinks(pupilData, timeListComplete):
    # filter any blinks and nan values lasting around 250ms (on average)
    # http://faculty.washington.edu/chudler/facts.html
   
    # blink is every nan value in the range of 100-400ms 
    # 250 ms (23 samples) before and after the blink will also be removed
    extraBlinkSamples = 23    
    
    # remove single missing data, that are due to hardware error
    missingVal_Single = np.argwhere(np.isnan(pupilData))
    missingVal_Single = list(itertools.chain.from_iterable(missingVal_Single)) # flatten the list
    missingVal_SingleDifference = [t - s for s, t in zip(missingVal_Single, missingVal_Single[1:])] # find difference 
    # between consecutive elements
    missingVal_SingleDifference.insert(0, missingVal_Single[0]) # insert the first blink index in the beginning of list
    
    # the list missingVal_SingleDifference contains the index of the first blink, followed by the difference in the index to 
    # the next nan value
    
    # first remove the single nan values, which are missing data
    eyeTracker_missingData = list() # list with index of single missing data  
    valInd = -1

    for val in missingVal_SingleDifference:
        valInd = valInd + 1
        if valInd == 0:
            continue
        if val != 1:
            if missingVal_SingleDifference[valInd-1] !=1: # if there are 2 consecutive missing values (denoted by 2 consecutive
                # non 1 numbers, they are added to the list of eyeTracker_missingData)
                eyeTracker_missingData.append(sum(missingVal_SingleDifference[:valInd]))
                
    # remove single missing values from pupil data
    pupilData_woSingleMissingData0 = [pupilData[ind] if ind not in eyeTracker_missingData else [] 
                                     for ind in range(0, len(pupilData))]
    pupilData_woSingleMissingData = [x for x in pupilData_woSingleMissingData0 if x]
    
    # remove the times for single missing values in pupil data
    timeList_woSingleMissingData0 = [timeListComplete[ind] if ind not in eyeTracker_missingData else [] 
                                     for ind in range(0, len(timeListComplete))]
    timeList_woSingleMissingData = [x for x in timeList_woSingleMissingData0 if x]
    
#     print(len(timeList_woSingleMissingData))
    
    
    
    
    # find the nan values again from pupilData_woSingleMissingData
    missingVal_Rest = np.argwhere(np.isnan(pupilData_woSingleMissingData))
    missingVal_Rest = list(itertools.chain.from_iterable(missingVal_Rest))
    missingVal_RestDifference = [t - s for s, t in zip(missingVal_Rest, missingVal_Rest[1:])]
    missingVal_RestDifference.insert(0, missingVal_Rest[0])
    
    
    # compile and create list of start and end of blinks
    blink_missingData = dict()
    blink_missingData['Start'] = list()
    blink_missingData['End'] = list()
    
    valInd = -1
    for val in missingVal_RestDifference:
        valInd = valInd + 1
        if val > 1:
            #print('value', val)
            # instead of appending the actual index of blink start, since 250ms before and after the blink need to be
            # removed, it is also appended here.
            
            # just make sure that the additional samples do not make the index of blink go in negative
            if sum(missingVal_RestDifference[:valInd+1])-extraBlinkSamples > 0:
                
                blink_missingData['Start'].append(sum(missingVal_RestDifference[:valInd+1])-extraBlinkSamples)
            else:
                blink_missingData['Start'].append(0)
            
            if valInd == 0:
                continue
                
            # make sure that the additional samples do not increase the index to more than the length of the pupilData
            if sum(missingVal_RestDifference[:valInd])+extraBlinkSamples < len(pupilData_woSingleMissingData):
                blink_missingData['End'].append(sum(missingVal_RestDifference[:valInd])+extraBlinkSamples)
            else:
                blink_missingData['End'].append(len(pupilData_woSingleMissingData)-1)
            #print('end', sum(missingVal_RestDifference[:valInd]))
      
    # add the last blink index
    # make sure that the additional samples do not increase the index to more than the length of the pupilData
    if sum(missingVal_RestDifference)+extraBlinkSamples < len(pupilData_woSingleMissingData):
        blink_missingData['End'].append(sum(missingVal_RestDifference)+extraBlinkSamples)
    else:
        blink_missingData['End'].append(len(pupilData_woSingleMissingData)-1)
      
    
    # print start and end values
    #for ind in range(0,len(blink_missingData['Start'])):
    #    print(blink_missingData['Start'][ind]+23, blink_missingData['End'][ind]-23)
        
        
        
    # need to create a list containing indexes that are to be removed
    blinkIndexList = list()
    
    #print(len(blink_missingData['Start']), len(blink_missingData['End']))
    
    
    # remove blinks and additional data from pupil data to get filtered data
    for indInd in range(0, len(blink_missingData['Start'])):
        blinkIndexList.append(range(blink_missingData['Start'][indInd], blink_missingData['End'][indInd]+1))
    # flatten the list
    blinkIndexList = list(itertools.chain.from_iterable(blinkIndexList))
    
    
    ##print(len(pupilData_woSingleMissingData))
    
    pupilData_woRestMissingData0 = [pupilData_woSingleMissingData[ind] if ind not in blinkIndexList else [] 
                                     for ind in range(0, len(pupilData_woSingleMissingData))]
    #for i in enumerate(pupilData_woRestMissingData0):
    #    print(i)
    pupilData_filter = [x for x in pupilData_woRestMissingData0 if x]
    
    #for i in enumerate(pupilData_filter):
    #    print(i)
        
    # remove the times for single missing values in pupil data
    timeList_woRestMissingData0 = [timeList_woSingleMissingData[ind] if ind not in blinkIndexList else [] 
                                     for ind in range(0, len(timeList_woSingleMissingData))]
    time_filter = [x for x in timeList_woRestMissingData0 if x]
    
    #print(len(pupilData_filter))
        
    if np.nan in pupilData_filter:
        print('nan values still present in pupil data')
        #for i in enumerate(pupilData_woSingleMissingData):
        #    print(i)
        
    
    return pupilData_filter, time_filter, blink_missingData

In [14]:
def modmax(d):
    # modulus maxima detection
    
    # compute signal modulus
    m = [0.0]*len(d)
    for i in range(0, len(d)):
        m[i] = math.fabs(d[i])
    
    # if value is larger than both neighbours , and strictly
    # larger than either , then it is a local maximum
    t = [0.0]*len(d)
    for i in range(0, len(d)):
        ll = m[i-1] if i >= 1 else m[i]
        oo = m[i]
        rr = m[i+1] if i < len(d)-2 else m[i]
        if (ll <= oo and oo >= rr) and (ll < oo or oo > rr):
            # compute magnitude
            t[i] = math.sqrt(d[i]**2)
        else:
            t[i] = 0.0
    #print(len(t))
    return t

In [15]:
def plotPupilSize(pupilData, timeData, TrialNumber):
    
    dataLenEqualizer = min(min(len(pupilData['Left']), len(pupilData['Right'])), len(timeData))
    
    fig = plt.figure()
    ax = fig.add_subplot(1,1,1)
    ax.plot(timeData[0:dataLenEqualizer], pupilData['Left'][0:dataLenEqualizer], 'b')
    ax.plot(timeData[0:dataLenEqualizer], pupilData['Right'][0:dataLenEqualizer], 'r')
    
    ax.set_ylabel('Absolute pupil size [in mm]')

    ax.set_title(TrialNumber)


In [16]:
def hampel(vals_orig, k, sd):
    '''
    vals: pandas series of values from which to remove outliers
    k: size of window (including the sample; 7 is equal to 3 on either side of value)
    '''
    # Obtained from: https://stackoverflow.com/questions/46819260/filtering-outliers-how-to-make-median-based-
    # hampel-function-faster
    
    #plt.plot(vals_orig)
    
    #Make copy so original not edited
    vals = pd.DataFrame(vals_orig)      
    #print(vals.isnull().any())
    vals0 = vals.replace([np.inf, -np.inf], np.nan)
    #vals = vals0.astype(float).fillna(method = 'backfill') # linear interpolation instead 
    #print(vals)
    vals = vals0.astype(float).interpolate('linear', limit_direction = 'both') # linear interpolation instead of 
    # simply copying the previous value --\ linear interpolation than cubic to not add any patterns in the data, limit direction
    # set to both, to interpolate the nan values occuring from the start of the series
    
    L= 1.4826
    rolling_median = vals.rolling(window=k, min_periods=1, center=True).median()
    
    #print(rolling_median)
    difference = np.abs(rolling_median-vals)
    median_abs_deviation = difference.rolling(k).median()
    threshold = sd * L * median_abs_deviation
    outlier_idx = difference>threshold
    vals[outlier_idx] = rolling_median[outlier_idx]
    #print(vals)
    #print('datatype', vals.dtypes)
    #print(vals.isnull().any())
    #vals.plot()
    return(vals)

In [17]:
def ipaFunc(d):
    # compute ipa value of pupil diameter
    IPA = list()
    #print(len(d.pupildata.values))
    # obtain 2-level DWT of pupil diameter signal d
    
    # get signal duration (in seconds)
    tt = ((d.timestamp.values[-1] - d.timestamp.values[0]).item())/1000000000

    
    
    try:
        (cA2,cD2,cD1) = pywt.wavedec(d.pupildata.values,'sym12','symmetric', 2)
    except ValueError:
        print('value error in wavedec')
        return
        
    # normalize by 1=2j , j = 2 for 2-level DWT
    cA2[:] = [x / math.sqrt(4.0) for x in cA2]
    cD1[:] = [x / math.sqrt(2.0) for x in cD1]
    cD2[:] = [x / math.sqrt(4.0) for x in cD2]
    
    # detect modulus maxima , see Listing 2
    cD2m = modmax(cD2)
    #print(len(cD2m))
    
    # threshold using universal threshold l_univ = s*sqrt(2logn)
    # where s is the standard deviation of the noise
    luniv = np.std(cD2m) * math.sqrt(2.0*np.log2(len(cD2m)))
    cD2t = pywt.threshold(cD2m ,luniv,mode="hard")
        
    # compute IPA
    ctr = 0
    for i in range(0, len(cD2t)):
        if math.fabs(cD2t[i]) > 0: ctr += 1
        #IPA = float(ctr)/tt
        # maybe each pupil data has an IPA?
    IPA = (float(ctr)/tt)
    
    return IPA, cD2m, cD2t, cD2, cD1, cA2, tt

In [18]:
def FindAndPlotPupilSizeForEpoch(GazeLog, TimeEpochTrial):
    # function that uses the list of start and end trial times to find the pupil sizes for those trials and plots them
    
    # first create a list of times in gaze log
    timeStrGazeLog = [item3[0] for item3 in GazeLog]
    # convert the list of strings to datetime formats
    timeGazeLog = timeConversion(timeStrGazeLog)
    
    # internal time, to depict seconds
    timeInternalGazeLog = [float(item3[1]) for item3 in GazeLog]
    
    # extract pupil sizes in decimals from the strange 2 columns for every pupil
    pupilLogL, pupilLogR = Convert2ColumnSizesTo1(GazeLog)
    
    ipaList = list()
    timeOfGaze_TrialList = list()
    
    # for every epoch, plot the pupil size
    for trialNr in range(0, len(timeEpochTrial['Start'])):
        if trialNr == 0:
            continue
        # find pupil sizes for the trial
        pupilSizeL_Trial, pupilSizeR_Trial, timeGaze_Trial, timeInternal_Trial = PupilSizeFromTrialTimes(
            [TimeEpochTrial['Start'][trialNr], TimeEpochTrial['End'][trialNr]], timeGazeLog, 
                                timeInternalGazeLog, pupilLogL, pupilLogR)
        
        pupilSize_Trial = dict()
        pupilSize_Filter = dict()
        pupilSize_woBlink = dict()
        
        # find difference in consecutive elements of internal time
        timeInternalDifference = [t - s for s, t in zip(timeInternal_Trial, timeInternal_Trial[1:])]
        # divide by 1000 to make it s
        timeOfGaze_Trial = [sum(timeInternalDifference[:i])/1000000 for i in range(1,len(timeInternalDifference))]

        # some trials were skipped, because the sentence was written before. If the time of trial is less than
        # 10s, the trial is skipped
        if timeOfGaze_Trial[-1] < 20:
            print('trial number ', trialNr+1, 'with', timeOfGaze_Trial[-1], 's will be skipped')
            continue
        
        pupilSize_Trial['Left'] = pupilSizeL_Trial
        pupilSize_Trial['Right'] = pupilSizeR_Trial
        
        #if trialNr == 4:
        #    for i in range(0, len(pupilSizeL_Trial)):
        #        print(pupilSizeL_Trial[i], pupilSizeR_Trial[i])
            
        #print('Trial', len(pupilSizeL_Trial), len(pupilSizeR_Trial))
        
        # filter the blinks
        pupilSizeL_woBlink, time_filter, missingPupilData = filterBlinks(
            pupilSizeL_Trial, timeGaze_Trial)
        
        pupilSizeR_woBlink, time_filter, missingPupilData = filterBlinks(
            pupilSizeR_Trial, timeGaze_Trial)
        

        
        #print(index_blinkEndL)
        #print(index_blinkEndR)
        pupilSize_woBlink['Left'] = pupilSizeL_woBlink
        pupilSize_woBlink['Right'] = pupilSizeR_woBlink
        
        #print('After blink', len(pupilSizeL_woBlink), len(pupilSizeR_woBlink))
        # Hampel filter to remove the outliers
        winSize = 25
        pupilSizeL_filter = hampel(pupilSizeL_woBlink, winSize, 3)
        pupilSizeR_filter = hampel(pupilSizeR_woBlink, winSize, 3)

        pupilSize_Filter['Left'] = pupilSizeL_filter.values.tolist()
        pupilSize_Filter['Right'] = pupilSizeR_filter.values.tolist()
        
        pupilSizeL_filterList = [i[0] for i in pupilSizeL_filter.values]
        pupilSizeR_filterList = [i[0] for i in pupilSizeR_filter.values]
        
        #print('filter', len(pupilSizeL_filterList), len(pupilSizeR_filterList))
        RLCorrelation = np.corrcoef(pupilSizeL_filterList, pupilSizeR_filterList)
        #print(RLCorrelation)
        
        pupilAvg = [((pupilSizeL_filterList[i] + pupilSizeR_filterList[i])/2) for i in range(0, min(len(pupilSizeL_filterList), len(pupilSizeR_filterList)))]
        
        pupilLog_filter_wTime_Tuple = list(zip(time_filter, pupilAvg))
        pupilAndTimeDf =  pd.DataFrame(pupilLog_filter_wTime_Tuple, columns=['timestamp','pupildata'])
        
        # compute IPA for the trial
        ipaVal, coeff_modmax, coeff_hard, coeff_D2, coeff_D1, coeff_A, timePeriodTrial = ipaFunc(pupilAndTimeDf)
        
        #print(trialNr+1, ipaVal, timeOfGaze_Trial[-1])
        
        ipaList.append(ipaVal)
        timeOfGaze_TrialList.append(timeOfGaze_Trial[-1])
        
    return ipaList, timeOfGaze_TrialList

In [179]:
subjName = r'C:\DTU\Data\201812_ExptToCheckMovementEffect\Data'
j = 0
flagFirstSubj = 0
pupilData = dict()
pupilData['RLCorrelation'] = []

# extract self-reported scores list and LIX score of given sentence
file_name = r'C:/DTU/Data/201812_ExptToCheckMovementEffect/Data/Scores.xlsx'

testNr = 'Test_wChinRest'
for root, dirs, subfolder in os.walk(subjName):
    if not dirs and testNr in root and 'ph' in root:
        
        if 'tb' in root or 'trial' in root:
            continue
            
        userKeys = None
        gazeLog = None
        keysSelected = None
        
        for file in subfolder:
            if fnmatch.fnmatch(file, 'user_looks*'):
                try:
                    
                    fUserKey = open(root + '\\' + file, encoding='utf-8')
                    readerUserKey = csv.reader(fUserKey)
                    userKeys = list(readerUserKey)
                    
                    userKeys.remove(userKeys[0])
                except:
                    if fUserKey is not None:
                        
                        fUserKey.close()
                    else:
                        print('error in opening the user looks at log file')
            
            elif fnmatch.fnmatch(file, 'KeySelection*'):
                try:
                    
                    fKeysSelected = open(root + '\\' + file, encoding='utf-8')
                    readerKeysSelected = csv.reader(fKeysSelected)
                    keysSelected = list(readerKeysSelected)
                    
                    keysSelected.remove(keysSelected[0])
                except:
                    if fKeysSelected is not None:
                        
                        fKeysSelected.close()
                    else:
                        print('error in opening the KeySelection log file')
            
            elif fnmatch.fnmatch(file, 'tobiiGazeLog*'):
                try:
                    fGazeLog = open(root + '\\' + file, encoding='utf-8')
                    readerGazeLog = csv.reader(fGazeLog)
                    gazeLog = list(readerGazeLog)
                    
                    gazeLog.remove(gazeLog[0]) # would not matter much even if the first row was not labels
                    gazeLog.remove(gazeLog[-1])

                except:
                    if fGazeLog is not None:
                        fGazeLog.close()
                    else:
                        print('error in opening the gaze log file')
            else:
                continue
            
                # if all these lists exist
            if userKeys is None or keysSelected is None or gazeLog is None:
                continue
            else:
                
                a = re.compile('(?<=ExptToCheckMovementEffect\\\\Data\\\\)(.*)(?=\\\\2018-1)')
                subjName = a.findall(root)[0]
                print(subjName)
                
                sheet_to_df_map = pd.read_excel(file_name, sheet_name=subjName[0:2])
                columnName = testNr + '_LIX'
                
                # find total of difficulty score
                scoresDifficulty = sheet_to_df_map[columnName]
                scoresDifficulty = scoresDifficulty[1:]

                # find start time of typing
                timeTyping = OptiKeyTypingTime(userKeys)
                
                # for some of the subjects, the data was not completely collected
                if subjName == 'sa\Test_woChinRest\p1' or subjName == 'rh\Test_wChinRest\p1':
                    del keysSelected[-1]
                
                # divide complete data into epochs of phrases
                timeStartEndMixed = FindTrialEndTimes(keysSelected, timeTyping)
                
                # create trial time epoch using the list of start/end times of trial and userKeys, to make sure that 
                # Sleep is completely there in every trial, to allow for baseline
                timeEpochTrial = CreateTimeEpochsOfTrials(timeStartEndMixed, userKeys)
                #print(timeEpochTrial)
                
                # find and plot pupil size for every trial
                ipaList, timeOfTrialList = FindAndPlotPupilSizeForEpoch(gazeLog, timeEpochTrial)
                
                if 'sa\Test_woChinRest' in subjName or 'rh\Test_wChinRest' in subjName:
                    if 'p1' in root:
                        ipaList1 = ipaList
                        timeOfTrialList1 = timeOfTrialList
                        #print('1', ipaList1)
                    else:
                        if 'sa\Test_woChinRest' in subjName:
                            ipaList2 = ipaList[1:]
                            timeOfTrialList2 = timeOfTrialList[1:]
                        else:
                            ipaList2 = ipaList
                            timeOfTrialList2 = timeOfTrialList
                        #print('2', ipaList2)
#                 for i in enumerate(ipaList):
#                     print(i)
#                 print(scoresDifficulty)   
                    

pt\Test_wChinRest


In [180]:
if 'sa\Test_woChinRest' in subjName or 'rh\Test_wChinRest' in subjName:
    print('Addition of lists')
    ipaList = ipaList1 + ipaList2
    timeOfTrialList = timeOfTrialList1 + timeOfTrialList2

timeOfTrialListPlot = [i/1000 for i in timeOfTrialList]

#print(ipaList)
#print(scoresDifficulty)
# divide scores as per difficulty

scoreDifficult = list()
scoreMedium = list()
scoreEasy = list()



for ind in range(0, len(scoresDifficulty)):
    #print(scoresDifficulty.values[ind])
    if scoresDifficulty.values[ind] > 2:
        scoreDifficult.append(ipaList[ind])
        ipaDifficult.append(ipaList[ind])
    elif scoresDifficulty.values[ind] > 1:
        scoreMedium.append(ipaList[ind])
        ipaMedium.append(ipaList[ind])
    else:
        scoreEasy.append(ipaList[ind])
        ipaEasy.append(ipaList[ind])
        

# fig = plt.figure()
# ax = fig.add_subplot(1,1,1)
# ax.plot(range(1,len(scoreDifficult)+1), scoreDifficult, 'r', marker = 'o', linestyle = '-', label = 'difficult')
# ax.plot(range(1,len(scoreMedium)+1), scoreMedium, 'b', marker = 's', linestyle = '-', label = 'medium')
# ax.plot(range(1,len(scoreEasy)+1), scoreEasy, 'c', marker = 'd', linestyle = '-', label = 'easy')

#ax.plot(range(1,len(scoresDifficulty)+1), scoresDifficulty/100, 'b', label = 'difficulty')
#ax.plot(range(1, len(timeOfTrialList)+1), timeOfTrialListPlot, 'c', label = 'time' )
# ax.set_title(subjName)
# ax.legend()
             
#print(ipaList)
#print(scoresDifficulty)
print(subjName)
print('Correlation:', np.corrcoef(ipaList,scoresDifficulty))
print('Correlation between time and ipa', np.corrcoef(ipaList, timeOfTrialList))
print('Correlation between time and difficulty', np.corrcoef(timeOfTrialList, scoresDifficulty))

pt\Test_wChinRest
Correlation: [[ 1.         -0.29496166]
 [-0.29496166  1.        ]]
Correlation between time and ipa [[ 1.         -0.49055908]
 [-0.49055908  1.        ]]
Correlation between time and difficulty [[1.         0.62724791]
 [0.62724791 1.        ]]


In [162]:
ipaMean = [ np.mean(ipaEasy), np.mean(ipaMedium) , np.mean(ipaDifficult)]
ipaStd = [ np.std(ipaEasy), np.std(ipaMedium) , np.std(ipaDifficult)]

In [163]:
fig = plt.figure()
ax = fig.add_subplot(1,1,1)
ax.errorbar([1, 2, 3], ipaMean, ipaStd, marker = 'o', capsize=4)
ax.set_title(subjName)
plt.xticks([1,2,3], ['Easy', 'Medium', 'Difficult'])

([<matplotlib.axis.XTick at 0x1f1a07676d8>,
 <a list of 3 Text xticklabel objects>)

In [147]:
print(len(ipaEasy), len(ipaMedium), len(ipaDifficult))

18 18 18


In [181]:
ipaDifficult = list()
ipaMedium = list()
ipaEasy = list()

In [113]:
print('Difficult', np.mean(ipaDifficult), np.std(ipaDifficult))
print('Medium', np.mean(ipaMedium), np.std(ipaMedium))
print('Easy', np.mean(ipaEasy), np.std(ipaEasy))

Difficult 0.060647291883884756 0.008377099231302008
Medium 0.11164444593303706 0.0
Easy 0.1017331417510663 0.04441789814682203


In [164]:
fig = plt.figure()
ax = fig.add_subplot(1,1,1)
ax.errorbar([1, 2, 3], ipaMean, ipaStd, marker = 'o', capsize=4)
plt.xticks([1,2,3], ['Easy', 'Medium', 'Difficult'])

([<matplotlib.axis.XTick at 0x1f1b350bef0>,
 <a list of 3 Text xticklabel objects>)